## Импортируем библиотеки

In [49]:
import pytz
import pandas as pd
import numpy as np
from plotly import graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image, PageBreak  # добавляем PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import io
import warnings
warnings.filterwarnings("ignore")

### Импорт данных – объединённые таблицы BotDialogueFormData,BotScenario,BotCompany

In [5]:
df = pd.read_csv('query_result_2025-01-27T11_45_49.077239+03_00.csv')

In [6]:
df.head(1)

,BotDialogueFormDataId,BotScenarioId,Phone,StartDate,FinishDate,DialogueId,FormData,OutgoingCallJobTryId,IncomingBotCallId,Created,CreatedBy,Updated,UpdatedBy,scenario_name,company_name
0,43185978,237,+79034499894,2024-04-16T12:30:21.097351,2024-04-16T12:30:58.743942,4fcaa6c6-accd-4d94-ae9d-9a88760b1d74,"{""in_container"":{""phone"":""+79034499894"",""kind""...",67932004.0,NaN,2024-04-16T12:31:01.282752,system,2024-04-16T12:31:01.282752,system,sbm_delay_api-test,СберМаркет


In [7]:
# Преобразуем строковые даты в datetime
df['StartDate'] = pd.to_datetime(df['StartDate'], format='ISO8601')
df['FinishDate'] = pd.to_datetime(df['FinishDate'], format='ISO8601')

In [8]:
# Базовая обработка данных
df['duration_seconds'] = (df['FinishDate'] - df['StartDate']).dt.total_seconds()
# df['company_name'] = np.where(df['company_name'].str.contains('Авито', case=False), 'Авито', df['company_name'])
# df['company_name'] = np.where(
#     df['company_name'].str.contains('сбермаркет|сбермаркет', case=False),
#     'Сбермаркет',
#     df['company_name']
# )
# df['company_name'] = np.where(
#     df['company_name'].str.contains('газпром', case=False),
#     'Газпром',
#     df['company_name']
# )
# df['company_name'] = np.where(
#     df['company_name'].str.contains('билайн', case=False),
#     'Билайн',
#     df['company_name']
# )
# df['company_name'] = np.where(
#     df['company_name'].str.contains('согаз|согаз мед|sogazmed', case=False),
#     'СОГАЗ-Мед',
#     df['company_name']
# )
df['call_type'] = np.where(df['OutgoingCallJobTryId'].isna(), 'Входящий', 'Исходящий')

In [9]:
# Добавляем временные метрики
df['hour'] = df['StartDate'].dt.hour
df['date'] = df['StartDate'].dt.date
df['month'] = df['StartDate'].dt.to_period('M')

In [16]:
# 1. KPI значения - общая длительность звонков
total_duration = df.groupby('call_type')['duration_seconds'].sum() / 60

# Топ 10 компаний входящей линии

In [18]:
def get_top_companies_in(df, sort_by='Входящий'):
    company_stats = df.pivot_table(
        index='company_name',
        columns='call_type',
        values='duration_seconds',
        aggfunc='sum',
        fill_value=0
    ) / 60
    return company_stats.nlargest(10, sort_by)

# Создаем таблицу топ-10 компаний по входящим
top_companies_in = get_top_companies_in(df)

# Топ 10 компаний исходящей линии

In [21]:
def get_top_companies_out(df, sort_by='Исходящий'):
    company_stats = df.pivot_table(
        index='company_name',
        columns='call_type',
        values='duration_seconds',
        aggfunc='sum',
        fill_value=0
    ) / 60
    return company_stats.nlargest(10, sort_by)

# Создаем таблицу топ-10 компаний по исходящим
top_companies_out = get_top_companies_out(df)

# Топ 10 компаний по динамике

In [24]:
def get_companies_dynamics(df):
    monthly_stats = df.pivot_table(
        index=['company_name', 'month'],
        values='duration_seconds',
        aggfunc='sum',
        fill_value=0
    ) / 60

    monthly_stats = monthly_stats.reset_index()
    monthly_stats = monthly_stats.sort_values(['company_name', 'month'])
    monthly_stats['change'] = monthly_stats.groupby('company_name')['duration_seconds'].pct_change()

    company_dynamics = monthly_stats.groupby('company_name')['change'].agg([
        ('avg_change', 'mean'),
        ('last_change', 'last'),
        ('total_minutes', lambda x: monthly_stats.loc[x.index, 'duration_seconds'].sum())
    ]).round(3)

    company_months = monthly_stats[monthly_stats['duration_seconds'] > 0].groupby('company_name').size()
    company_dynamics['active_months'] = company_months

    top_dynamics = company_dynamics.nlargest(10, 'avg_change')
    top_dynamics['avg_change'] = (top_dynamics['avg_change'] * 100).round(1).astype(str) + '%'
    top_dynamics['last_change'] = (top_dynamics['last_change'] * 100).round(1).astype(str) + '%'
    
    return top_dynamics

# Получаем топ-10 по динамике
top_dynamics = get_companies_dynamics(df)

In [26]:
# 3. Ключевые метрики
metrics = pd.DataFrame()
# Временные метрики
calls_by_hour = df.groupby('hour').size()
calls_by_date = df.groupby('date').size()
calls_by_month = df.groupby('month').size()

metrics.loc['Среднее количество звонков в час', 'Значение'] = calls_by_hour.mean().round(2)
metrics.loc['Среднее количество звонков в день', 'Значение'] = calls_by_date.mean().round(2)
metrics.loc['Среднее количество звонков в месяц', 'Значение'] = calls_by_month.mean().round(2)

# Одновременные звонки
def count_concurrent_calls(group):
    group = group.dropna(subset=['StartDate', 'FinishDate'])
    if len(group) == 0:
        return 0
    
    timeline = pd.DataFrame()
    for _, call in group.iterrows():
        idx = pd.date_range(start=call['StartDate'], end=call['FinishDate'], freq='1min')
        timeline = pd.concat([timeline, pd.DataFrame(index=idx)])
    return timeline.groupby(level=0).size().max()

concurrent_calls = df.groupby(df['StartDate'].dt.date).apply(count_concurrent_calls)
metrics.loc['Максимальное количество одновременных звонков', 'Значение'] = concurrent_calls.max()


In [27]:
# 4. Распределение по времени суток
hourly_distribution = (df.groupby('hour').size() / len(df) * 100).round(2)

In [28]:
# Создаем график распределения по времени суток
def create_hourly_distribution_chart(hourly_distribution):
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=hourly_distribution.index,
        y=hourly_distribution.values,
        name='Распределение звонков',
        marker_color='rgb(26, 118, 255)'
    ))
    
    fig.update_layout(
        title='Распределение звонков по времени суток',
        xaxis_title='Час',
        yaxis_title='Процент звонков (%)',
        showlegend=True,
        template='plotly_white',
        height=600
    )
    
    return fig

In [29]:
# Создаем график последних 60 дней
def create_last_60_days_chart(df):
    last_date = df['StartDate'].max()
    start_date = last_date - timedelta(days=60)
    
    mask = (df['StartDate'] >= start_date) & (df['StartDate'] <= last_date)
    last_60_days_df = df[mask]
    
    daily_duration = last_60_days_df.groupby(last_60_days_df['StartDate'].dt.date)['duration_seconds'].sum() / 60
    
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=daily_duration.index,
        y=daily_duration.values,
        name='Длительность звонков',
        marker_color='rgb(55, 83, 109)'
    ))
    
    fig.update_layout(
        title='Длительность звонков за последние 60 дней',
        xaxis_title='Дата',
        yaxis_title='Длительность (минуты)',
        showlegend=True,
        template='plotly_white',
        height=600
    )
    
    return fig

In [30]:
# Создаем графики
hourly_fig = create_hourly_distribution_chart(hourly_distribution)
last_60_days_fig = create_last_60_days_chart(df)

In [31]:
# Регистрируем шрифт
pdfmetrics.registerFont(TTFont('CustomFont', 'fonts/dv.ttf'))

# Генерация отчёта

In [51]:
def create_pdf_report(df, metrics, top_companies_in, top_companies_out, top_dynamics, hourly_distribution, hourly_fig, last_60_days_fig, total_duration):
    doc = SimpleDocTemplate(
        "analysis_report.pdf",
        pagesize=letter,
        rightMargin=72,
        leftMargin=72,
        topMargin=72,
        bottomMargin=72
    )
    
    elements = []
    
    styles = getSampleStyleSheet()
    title_style = styles['Heading1']
    subtitle_style = styles['Heading2']
    normal_style = styles['Normal']
    
    title_style.fontName = 'CustomFont'
    subtitle_style.fontName = 'CustomFont'
    normal_style.fontName = 'CustomFont'
    
    moscow_tz = pytz.timezone('Europe/Moscow')
    moscow_time = datetime.now(moscow_tz)
    elements.append(Paragraph("Отчет по анализу звонков", title_style))
    elements.append(Paragraph(f"Обновление отчёта: {moscow_time.strftime('%d.%m.%Y %H:%M')} (МСК)", normal_style))
    elements.append(Spacer(1, 20))
    
    # Ключевые метрики
    elements.append(Paragraph("Ключевые показатели", subtitle_style))
    key_metrics_data = [
        ["Показатель", "Значение"],
        ["Общая длительность входящей линии", f"{total_duration['Входящий']:.2f} минут"],
        ["Общая длительность исходящей линии", f"{total_duration['Исходящий']:.2f} минут"],
        ["Среднее количество звонков в день", f"{metrics.loc['Среднее количество звонков в день', 'Значение']}"],
        ["Максимальное количество одновременных звонков", f"{metrics.loc['Максимальное количество одновременных звонков', 'Значение']}"]
    ]
    
    t = Table(key_metrics_data, colWidths=[4*inch, 2*inch])
    t.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, -1), 'CustomFont'),
        ('FONTSIZE', (0, 0), (-1, 0), 14),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))
    elements.append(t)
    elements.append(Spacer(1, 20))
    # Добавляем график распределения по времени
    img_bytes = io.BytesIO()
    hourly_fig.write_image(img_bytes, format='png')
    img_bytes.seek(0)
    img = Image(img_bytes, width=6*inch, height=4*inch)
    elements.append(Paragraph("Распределение звонков по времени суток", subtitle_style))
    elements.append(img)
    elements.append(Spacer(1, 20))
    
    elements.append(PageBreak())
    # Таблица топ компаний по входящему трафику
    elements.append(Paragraph("Топ-10 компаний – входящий трафик", subtitle_style))
    companies_in_data = [["Компания", "Входящие (мин)", "Исходящие (мин)"]]
    for company in top_companies_in.index:
        companies_in_data.append([
            company,
            f"{top_companies_in.loc[company, 'Входящий']:.2f}",
            f"{top_companies_in.loc[company, 'Исходящий']:.2f}"
        ])
    
    t = Table(companies_in_data, colWidths=[3*inch, 2*inch, 2*inch])
    t.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, -1), 'CustomFont'),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))
    elements.append(t)
    elements.append(Spacer(1, 20))

    # Таблица топ компаний по исходящему трафику
    elements.append(Paragraph("Топ-10 компаний – исходящий трафик", subtitle_style))
    companies_out_data = [["Компания", "Входящие (мин)", "Исходящие (мин)"]]
    for company in top_companies_out.index:
        companies_out_data.append([
            company,
            f"{top_companies_out.loc[company, 'Входящий']:.2f}",
            f"{top_companies_out.loc[company, 'Исходящий']:.2f}"
        ])
    
    t = Table(companies_out_data, colWidths=[3*inch, 2*inch, 2*inch])
    t.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, -1), 'CustomFont'),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))
    elements.append(t)
    elements.append(Spacer(1, 20))
    elements.append(PageBreak())
    # Таблица топ компаний по динамике
    elements.append(Paragraph("Топ-10 компаний по динамике", subtitle_style))
    dynamics_data = [["Компания", "Среднее изменение", "Последнее изменение", "Всего минут", "Активных месяцев"]]
    for company in top_dynamics.index:
        dynamics_data.append([
            company,
            top_dynamics.loc[company, 'avg_change'],
            top_dynamics.loc[company, 'last_change'],
            f"{top_dynamics.loc[company, 'total_minutes']:.2f}",
            str(top_dynamics.loc[company, 'active_months'])
        ])
    
    t = Table(dynamics_data, colWidths=[2*inch, 1.5*inch, 1.5*inch, 1.5*inch, 1.5*inch])
    t.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, -1), 'CustomFont'),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))
    elements.append(t)
    elements.append(Spacer(1, 20))

    # Добавляем график последних 60 дней
    last_60_days_bytes = io.BytesIO()
    last_60_days_fig.write_image(last_60_days_bytes, format='png')
    last_60_days_bytes.seek(0)
    last_60_days_img = Image(last_60_days_bytes, width=6*inch, height=4*inch)
    elements.append(Paragraph("Длительность звонков за последние 60 дней", subtitle_style))
    elements.append(last_60_days_img)
    
    # Генерируем PDF
    doc.build(elements)

# Вызываем функцию с обновленными параметрами
create_pdf_report(df, metrics, top_companies_in, top_companies_out, top_dynamics, hourly_distribution, hourly_fig, last_60_days_fig, total_duration)